In [ ]:
pip install PyPDF2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd

In [ ]:
import PyPDF2
import csv

def parse_pdf(file_path):
    pdf = PyPDF2.PdfReader(file_path)

    data = []
    current_section = ""
    current_subsection = ""
    current_text = ""

    for page_num in range(len(pdf.pages)):
        page = pdf.pages[page_num]
        content = page.extract_text()

        lines = content.split('\n')

        for line in lines:
            line = line.strip()
            if line:
                if line.isupper():  # Identifying section headings
                    current_section = line
                    current_subsection = ""
                    current_text = ""
                elif line.islower():  # Identifying subsection headings
                    current_subsection = line
                else:
                    current_text += line + " "

                if current_text and (current_section or current_subsection):
                    data.append([current_section, current_subsection, current_text.strip()])

    return data

def write_to_csv(data, output_file):
    with open(output_file, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile, quoting=csv.QUOTE_NONE,  delimiter='|', quotechar='')
        writer.writerow(['Section', 'Subsection', 'Text'])
        writer.writerows(data)

# Usage
pdf_file = '/content/2005.14165.pdf'


output_excel_file = 'output.xlsx'
parsed_data = parse_pdf(pdf_file)

df = pd.DataFrame(parsed_data, columns=['Section', 'Subsection', 'Text'])

# Create an Excel workbook
workbook = Workbook()
sheet = workbook.active


# Clean and write the data to the sheet
for row in df.itertuples(index=False):
    try:
      cleaned_row = [cell.encode('utf-8', 'ignore').decode('utf-8') if cell else "" for cell in row]
      sheet.append(cleaned_row)
    except:
      pass

# Save the workbook to Excel file
workbook.save(output_excel_file)

print("Excel file created successfully!")







Excel file created successfully!


In [ ]:
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl import Workbook
import re

In [1]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 59.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 127.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 94.0 MB/s eta 0:00:00


In [2]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

In [3]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

tokenizer.add_special_tokens({'pad_token': "<pad>", 'bos_token':"<startofstring>", 'eos_token':'<endofstring>'})
tokenizer.add_tokens(["<bot>"])

model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))

Embedding(50261, 768)

In [4]:
model.generate(**tokenizer("I love cricket", return_tensors = "pt"))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


tensor([[   40,  1842, 18836, 50257, 50257, 50257, 50257, 50257, 50257, 50257,
         50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257]])

In [5]:
tokenizer("I love cricket", return_tensors = "pt")

{'input_ids': tensor([[   40,  1842, 18836]]), 'attention_mask': tensor([[1, 1, 1]])}

In [6]:
tokenizer.decode(model.generate(**tokenizer("I love cricket, but I don't play itb now", return_tensors="pt"))[0])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


"I love cricket, but I don't play itb now <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>"

In [7]:
model.num_parameters

<bound method ModuleUtilsMixin.num_parameters of GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50261, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50261, bias=False)
)>

In [34]:
from torch.utils.data import Dataset
import json
#from ChatData import ChatData

In [41]:
class ChatData(Dataset):
  def __init__(self, path:str, tokenizer):
      self.data = json.load(open(path, "r"))

      self.X = []
      for i in self.data:
          for j in i['dialog']:
              self.X.append(j['text'])

      for idx, i in enumerate(self.X):
          try:
              self.X[idx] = "<startofstring> "+i+" <bot>: "+self.X[idx+1]+" <endofstring>"
          except:
              break

      self.X = self.X[:20000]
      
      print(self.X[2])

      self.X_encoded = tokenizer(self.X,max_length=40, truncation=True, padding="max_length", return_tensors="pt")
      self.input_ids = self.X_encoded['input_ids']
      self.attention_mask = self.X_encoded['attention_mask']

  def __len__(self):
    pass

  def __getitem__(self):
    return(self.input[idx], self.attention_mas[idx])

In [42]:
chatData = ChatData("/content/drive/MyDrive/Colab Notebooks/chat_data.json", tokenizer)

<startofstring> I am a college student and i am a college student <bot>: I am go to gym and live on donations <endofstring>


In [37]:
chatData

In [38]:
from torch.optim import Adam

In [13]:
optim = Adam(model.parameters())

In [14]:


optim

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.001
    maximize: False
    weight_decay: 0
)

In [15]:
model.train()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50261, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50261, bias=False)
)

In [16]:
from torch.utils.data import DataLoader
import tqdm

In [45]:
chatData = DataLoader(chatData, batch_size = 64)

In [46]:
def train(ChatData,model,optim):
  epochs = 10

  for i in tqdm(range(epochs)):
    for X, a in chatData:
      loss = model(X,attention_mask=a, labels=X).loss
      loss.backward()
      optim.step()
    torch.save(model.state_dict(), "model_state.pt")

In [47]:
def infer(inp):
  inp = "<startofstring>"+inp+"<bot>"
  inp = tokenizer(inp)
  output = model.generate(**inp)
  output = tokenizer.decode(output[0])

In [48]:
from torch.utils.data import Dataset
import json

class ChatData(Dataset):
    def __init__(self, path:str, tokenizer):
        self.data = json.load(open(path, "r"))

        self.X = []
        for i in self.data:
            for j in i['dialog']:
                self.X.append(j['text'])

        for idx, i in enumerate(self.X):
            try:
                self.X[idx] = "<startofstring> "+i+" <bot>: "+self.X[idx+1]+" <endofstring>"
            except:
                break

        self.X = self.X[:10000]
        
        print(self.X[1])

        self.X_encoded = tokenizer(self.X,max_length=40, truncation=True, padding="max_length", return_tensors="pt")
        self.input_ids = self.X_encoded['input_ids']
        self.attention_mask = self.X_encoded['attention_mask']

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return (self.input_ids[idx], self.attention_mask[idx])

In [49]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from torch.optim import Adam
from torch.utils.data import DataLoader
import tqdm
import torch

def train(chatData, model, optim):

    epochs = 24

    for i in tqdm.tqdm(range(epochs)):
        for X, a in chatData:
            X = X.to(device)
            a = a.to(device)
            optim.zero_grad()
            loss = model(X, attention_mask=a, labels=X).loss
            loss.backward()
            optim.step()
        torch.save(model.state_dict(), "model_state.pt")
        print(infer("hello how are you"))

def infer(inp):
    inp = "<startofstring> "+inp+" <bot>: "
    inp = tokenizer(inp, return_tensors="pt")
    X = inp["input_ids"].to(device)
    a = inp["attention_mask"].to(device)
    output = model.generate(X, attention_mask=a )
    output = tokenizer.decode(output[0])
    return output


device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.add_special_tokens({"pad_token": "<pad>", 
                                "bos_token": "<startofstring>",
                                "eos_token": "<endofstring>"})
tokenizer.add_tokens(["<bot>:"])

model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))

model = model.to(device)

# print(tokenizer.decode(model.generate(**tokenizer("hey i was good at basketball but ",
#                          return_tensors="pt"))[0]))

chatData = ChatData("/content/drive/MyDrive/Colab Notebooks/chat_data.json", tokenizer)
chatData =  DataLoader(chatData, batch_size=64)

model.train()

optim = Adam(model.parameters(), lr=1e-3)

print("training .... ")
train(chatData, model, optim)



<startofstring> Thats good for you, i'm not very into new tech <bot>: I am a college student and i am a college student <endofstring>
training .... 


  4%|▍         | 1/24 [00:41<16:04, 41.94s/it]

<startofstring> hello how are you <bot>:  am am am am am am am am a am am am am am


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
  8%|▊         | 2/24 [01:20<14:39, 39.99s/it]

<startofstring> hello how are you <bot>: Hi <bot>: Hi <bot>: <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>


...

<startofstring> hello how are you <bot>:? <endofstring> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
 96%|█████████▌| 23/24 [15:28<00:40, 40.22s/it]

<startofstring> hello how are you <bot>: Hey, are you here? What's up? <endofstring> <pad> <pad> <pad>


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
100%|██████████| 24/24 [16:06<00:00, 40.27s/it]

<startofstring> hello how are you <bot>: hi <endofstring> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>


In [51]:
inp = "How is the weather today? "

In [52]:
infer(inp)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'<startofstring> How is the weather today? <bot>: Yes, it is going good <endofstring> <pad> <pad> <pad> <pad> <pad>'

In [53]:
inp = "What are your hobbies?"

In [54]:
infer(inp)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'<startofstring> What are your hobbies? <bot>: I am dealing with flower and garden work <endofstring> <pad> <pad> <pad> <pad>'

In [55]:
infer("What is your favourute color?")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'<startofstring> What is your favourute color? <bot>: I am not sure what i am a big fan of'

In [56]:
infer("What do you do for living?")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'<startofstring> What do you do for living? <bot>: I am a high school student and i am a college'

In [57]:
infer("What's your name?")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


"<startofstring> What's your name? <bot>: He'S 59th <endofstring> <pad> <pad> <pad> <pad> <pad> <pad> <pad>"

In [58]:
infer("do you like to play basketball?")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'<startofstring> do you like to play basketball? <bot>: I do not like to play. <endofstring> <pad> <pad> <pad>'

In [59]:
infer("do you know how much is 1+1?")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'<startofstring> do you know how much is 1+1? <bot>: i am a very experienced, i know'

In [60]:
infer("how many hours in a day?")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'<startofstring> how many hours in a day? <bot>: I am a full time student <endofstring> <pad> <pad> <pad> <pad>'

In [61]:
infer("how are you doing?")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'<startofstring> how are you doing? <bot>: i am a huge gamer, my mom is a very good person'